## 人均营收 ab test 样本测算

In [4]:
# --------------- SEP ---------------
price_power_analysis <- 
    function(prices, conversion_rates, mean_lift, sig_level = .1, power = .8) {
    # monte carlo settings
    SAMPLE_SIZE = 100000
    price_vector = c(0, prices)
    conversion_vector = c(1 - sum(conversion_rates), conversion_rates)
    # simulate sales for a population
    sales_vector = sample(price_vector, SAMPLE_SIZE, replace=TRUE, prob=conversion_vector)

    # assumed mean and standard deviation
    sample_mean = mean(sales_vector)
    sample_sd = sd(sales_vector)
    cat("营收平均值:", sample_mean, '\n')
    cat("营收标准差:", sample_sd, '\n')
    cat(rep('-', 20), '\n')

    # d: cohen's D
    power.t.test(d = mean_lift/sample_sd, sig.level = sig_level, power = power, alternative = "one.sided")
}

In [22]:
price_power_analysis(c(898, 2499), c(0.10, 0.01), 5)

营收平均值: 114.5284 
营收标准差: 359.9722 
- - - - - - - - - - - - - - - - - - - - 



     Two-sample t test power calculation 

              n = 46730.72
          delta = 0.01388996
             sd = 1
      sig.level = 0.1
          power = 0.8
    alternative = one.sided

NOTE: n is number in *each* group


## 转化率 ab test 样本测算

In [2]:
baseline = 0.085
relative_mde = 0.13
power.prop.test(p1 = baseline, p2 = baseline * (1 + relative_mde), sig.level = 0.1, power = 0.8, alternative = 'one.sided')


     Two-sample comparison of proportions power calculation 

              n = 6078.152
             p1 = 0.085
             p2 = 0.09605
      sig.level = 0.1
          power = 0.8
    alternative = one.sided

NOTE: n is number in *each* group


In [6]:
baseline = 0.085
abs_mde = 0.015
power.prop.test(p1 = baseline, p2 = baseline + abs_mde, sig.level = 0.1, power = 0.8, alternative = 'one.sided')


     Two-sample comparison of proportions power calculation 

              n = 3362.723
             p1 = 0.085
             p2 = 0.1
      sig.level = 0.1
          power = 0.8
    alternative = one.sided

NOTE: n is number in *each* group


In [170]:
prop.test(x = c(.5582 * 200, .4565 * 2000), n = c(200, 2000), alternative = "two.sided")


	2-sample test for equality of proportions with continuity correction

data:  c(0.5582 * 200, 0.4565 * 2000) out of c(200, 2000)
X-squared = 7.1544, df = 1, p-value = 0.007478
alternative hypothesis: two.sided
95 percent confidence interval:
 0.02674673 0.17665327
sample estimates:
prop 1 prop 2 
0.5582 0.4565 


## t test

In [127]:
require(dplyr)
require(readxl)
require(ggplot2)

options(repr.plot.width=12, repr.plot.height=6)

# helpers
save_png = function(plt, dir = "./",width = 1800,height = 800, res = 220) {
  path = paste0(dir,deparse(substitute(plt)),".png")
  message(path)
  png(path, width = width, height = height, res = res)
  plot(plt)
  dev.off()
}

In [147]:
df_ab = readr::read_csv(
    './input/ez_eo_直投_营收记录_20200901_20200915.csv', 
    col_types = cols(.default = "?", revenue_d25 = "i", year_revenue_d25 = "i", hiarpu_revenue_d25 = "i"),
    na = 'NULL',
    locale = readr::locale(encoding = "GBK"),
) %>% mutate(
    zt_first_real_price = zt_first_real_price / 100,
    year_revenue_d25 = year_revenue_d25 / 100,
    hiarpu_revenue_d25 = hiarpu_revenue_d25/ 100,
    total_revenue = zt_first_real_price + year_revenue_d25 + hiarpu_revenue_d25,
    proficient_level_name = ifelse(is.na(proficient_level_name), '未测试', proficient_level_name)
)

In [164]:
# 入群率
plot_d0_in_group_rate_eo_vs_ez = df_ab %>% group_by(activation_date, app_name) %>%
    summarise(
        users = n(),
        ratio_in_group = sum(in_group)/n(),
        total_revenue = mean(total_revenue, na.rm = T),
        year_revenue_d25 = mean(year_revenue_d25, na.rm = T),
        hiarpu_revenue_d25 = mean(hiarpu_revenue_d25, na.rm = T),
        .groups = 'drop',
    ) %>% ggplot(aes(x = activation_date, y = ratio_in_group, color = app_name)) +
    geom_line() +
    ylim(0, 1) +
    labs(title = '直投用户 D0 入群率 EO vs. EZ', subtitle = '20200901~0918', y = '')

save_png(plot_d0_in_group_rate_eo_vs_ez, res = 180)

./plot_d0_in_group_rate_eo_vs_ez.png



png 
  2

In [165]:
# 入群率，按等级
plot_d0_in_group_rate_eo_vs_ez_by_level = df_ab %>% group_by(activation_date, app_name, proficient_level_name) %>%
    summarise(
        users = n(),
        ratio_in_group = sum(in_group)/n(),
        total_revenue = mean(total_revenue, na.rm = T),
        year_revenue_d25 = mean(year_revenue_d25, na.rm = T),
        hiarpu_revenue_d25 = mean(hiarpu_revenue_d25, na.rm = T),
        .groups = 'drop',
    ) %>% ggplot(aes(x = activation_date, y = ratio_in_group, color = app_name)) +
    geom_line() +
    facet_wrap(proficient_level_name~.) +
    labs(title = '按等级直投用户 D0 入群率 EO vs. EZ', subtitle = '20200901~0918', y = '')

save_png(plot_d0_in_group_rate_eo_vs_ez_by_level, res = 180)

./plot_d0_in_group_rate_eo_vs_ez_by_level.png



png 
  2

In [168]:
# 测算
df_ab %>% group_by(app_name, proficient_level_name) %>%
    summarise(
        users = n(),
        ratio_in_group = sum(in_group)/n(),
        total_revenue = mean(total_revenue, na.rm = T),
        year_revenue_d25 = mean(year_revenue_d25, na.rm = T),
        hiarpu_revenue_d25 = mean(hiarpu_revenue_d25, na.rm = T),
        .groups = 'drop',
    )

app_name,proficient_level_name,users,ratio_in_group,total_revenue,year_revenue_d25,hiarpu_revenue_d25
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
eo,A1,780,0.8961538,220.11026,85.771795,115.338462
eo,A2,645,0.9162791,229.58295,102.099225,108.483721
eo,B1,664,0.9337349,312.09488,130.509036,162.585843
eo,B2,269,0.7249071,82.14126,16.691450,46.449814
eo,C1,53,0.7169811,19.00000,0.000000,0.000000
eo,C2,5,0.2000000,19.00000,0.000000,0.000000
eo,未测试,705,0.2269504,23.81844,1.273759,3.544681
ez,A0-,234,0.8931624,175.09402,49.170940,106.794872
ez,A0+,85,0.9647059,138.64706,119.647059,0.000000


In [129]:
# 群内 & 群外
df_ab %>% group_by(app_name) %>%
    summarise(
        users = n(),
        ratio_in_group = sum(in_group)/n(),
        total_revenue = mean(total_revenue, na.rm = T),
        year_revenue_d25 = mean(year_revenue_d25, na.rm = T),
        hiarpu_revenue_d25 = mean(hiarpu_revenue_d25, na.rm = T),
    )

`summarise()` ungrouping output (override with `.groups` argument)



app_name,users,ratio_in_group,total_revenue,year_revenue_d25,hiarpu_revenue_d25
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
eo,3121,0.7382249,181.6687,72.02884,90.63986
ez,5390,0.8571429,182.4408,79.37639,84.04213


In [130]:
# 群内
df_ab %>% filter(in_group == 1) %>%
    group_by(app_name) %>%
    summarise(
        users = n(),
        ratio_in_group = sum(in_group)/n(),
        total_revenue = mean(total_revenue, na.rm = T),
        year_revenue_d25 = mean(year_revenue_d25, na.rm = T),
        hiarpu_revenue_d25 = mean(hiarpu_revenue_d25, na.rm = T),
    )

`summarise()` ungrouping output (override with `.groups` argument)



app_name,users,ratio_in_group,total_revenue,year_revenue_d25,hiarpu_revenue_d25
<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
eo,2304,1,234.6285,93.93229,121.69618
ez,4620,1,208.3286,91.27517,98.03399


In [135]:
t.test(
    x = df_ab %>% filter(app_name == 'ez') %>% .$hiarpu_revenue_d25, 
    y = df_ab %>% filter(app_name == 'eo') %>% .$hiarpu_revenue_d25, 
    alternative = "two.sided",
)


	Welch Two Sample t-test

data:  df_ab %>% filter(app_name == "ez") %>% .$hiarpu_revenue_d25 and df_ab %>% filter(app_name == "eo") %>% .$hiarpu_revenue_d25
t = -0.63532, df = 6320.8, p-value = 0.5252
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -26.95574  13.76028
sample estimates:
mean of x mean of y 
 84.04213  90.63986 


In [136]:
t.test(
    x = df_ab %>% filter(app_name == 'ez', in_group == 1) %>% .$hiarpu_revenue_d25, 
    y = df_ab %>% filter(app_name == 'eo', in_group == 1) %>% .$hiarpu_revenue_d25, 
    alternative = "two.sided",
)


	Welch Two Sample t-test

data:  df_ab %>% filter(app_name == "ez", in_group == 1) %>% .$hiarpu_revenue_d25 and df_ab %>% filter(app_name == "eo", in_group == 1) %>% .$hiarpu_revenue_d25
t = -1.7788, df = 4204.1, p-value = 0.07534
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -49.741256   2.416875
sample estimates:
mean of x mean of y 
 98.03399 121.69618 


In [138]:
plot_d25_rev_per_new_user_eo_vs_ez = df_ab %>% 
    group_by(activation_date,app_name) %>%
    summarise(
        users = n(),
        ratio_in_group = sum(in_group)/n(),
        total_revenue = mean(total_revenue, na.rm = T),
        year_revenue_d25 = mean(year_revenue_d25, na.rm = T),
        hiarpu_revenue_d25 = mean(hiarpu_revenue_d25, na.rm = T),
    ) %>% ggplot(aes(x = activation_date, y = total_revenue, color = app_name)) +
    geom_line() +
    labs(title = 'D25 revenue per group user EO vs. EZ', subtitle = '20200901~0917', y = '')

save_png(plot_d25_rev_per_new_user_eo_vs_ez)

`summarise()` regrouping output by 'activation_date' (override with `.groups` argument)

./plot_d25_rev_per_new_user_eo_vs_ez.png



png 
  2

In [137]:
plot_d25_rev_per_group_user_eo_vs_ez = df_ab %>% 
    filter(in_group == 1) %>%
    group_by(activation_date,app_name) %>%
    summarise(
        users = n(),
        ratio_in_group = sum(in_group)/n(),
        total_revenue = mean(total_revenue, na.rm = T),
        year_revenue_d25 = mean(year_revenue_d25, na.rm = T),
        hiarpu_revenue_d25 = mean(hiarpu_revenue_d25, na.rm = T),
    ) %>% ggplot(aes(x = activation_date, y = total_revenue, color = app_name)) +
    geom_line() +
    labs(title = 'D25 revenue per group user EO vs. EZ', subtitle = '20200901~0917', y = '')

save_png(plot_d25_rev_per_group_user_eo_vs_ez)

`summarise()` regrouping output by 'activation_date' (override with `.groups` argument)

./plot_d25_rev_per_group_user_eo_vs_ez.png



png 
  2

直投 ROI 置信区间

In [77]:
roi_generator = function(size, reps=1000) {
    roi = vector()
    for (i in 1:reps) {
        # 体验课价格
        price_trial = 1
        # 转化价格分布，0 表示不卖
        price_vip = c(0, 720)
        # 转化概率分布 sum = 1
        conversion_vip = c(.928, .072)
        # 目标 CPA
        cpa_mean = 85
        # !!! 假设的 CPA 标准差，代表 CPA 的稳定性
        cpa_sd = 0
        # price and cost for each user
        price_vector = price_trial + sample(x = price_vip, prob = conversion_vip, size = size, replace = T)
        cost_vector = rnorm(n = size, mean = cpa_mean, sd = cpa_sd)      
        
        roi = c(roi, sum(price_vector)/sum(cost_vector))
    }
    roi
}

In [78]:
for (s in seq(from = 1000, to = 10000, by = 1000)) {
    qs = quantile(x = roi_generator(s), probs = c(.05, .95))
    cat('累计新增: ', s, '\n')
    cat('D8 ROI 置信区间: ', qs, '\n')
}

累计新增:  1000 
D8 ROI 置信区间:  0.5115294 0.7402353 
累计新增:  2000 
D8 ROI 置信区间:  0.5369412 0.7065647 
累计新增:  3000 
D8 ROI 置信区间:  0.5567059 0.6922353 
累计新增:  4000 
D8 ROI 置信区间:  0.5665882 0.6809412 
累计新增:  5000 
D8 ROI 置信区间:  0.5741271 0.6708612 
累计新增:  6000 
D8 ROI 置信区间:  0.5736471 0.6668235 
累计新增:  7000 
D8 ROI 置信区间:  0.5805042 0.6652101 
累计新增:  8000 
D8 ROI 置信区间:  0.5824706 0.6597647 
累计新增:  9000 
D8 ROI 置信区间:  0.584 0.6574588 
累计新增:  10000 
D8 ROI 置信区间:  0.5860706 0.6572235 


## 转化率的置信区间

$\text{var}(x_i) = p(1-p)$

$X = \frac{1}{N} \sum_i x_i$

$\text{sd}(X) = \frac{p(1-p)}{\sqrt{N}}$

通过置信度，按标准正态分布选取分位数，乘以 $\text{sd}(X)$ 作为置信区间宽度。

In [5]:
p = 0.09 # 基准群内转化率
N = 14000 * .10 # D0 入群人数
conf = 0.9 # 置信度

qnorm(1 - (1 - conf) / 2) * p * (1 - p) / sqrt(N) * 2

[1] 0.00720074

In [23]:
p = 0.75 # 基准群内转化率
N = 1200 # D0 入群人数
conf = 0.9 # 置信度

qnorm(1 - (1 - conf) / 2) * p * (1 - p) / sqrt(N)

[1] 0.008903031

In [27]:
sqrt((1 - 0.1)^2*0.4 + (0 - 0.1)^2*0.3 + (0 - 0.1)^2*0.3)/sqrt(100)*2

[1] 0.1148913

## prop test

In [2]:

prop.test(n = c(1988, 2537), x = c(171, 243))


	2-sample test for equality of proportions with continuity correction

data:  c(171, 243) out of c(1988, 2537)
X-squared = 1.1642, df = 1, p-value = 0.2806
alternative hypothesis: two.sided
95 percent confidence interval:
 -0.027039121  0.007506474
sample estimates:
    prop 1     prop 2 
0.08601610 0.09578242 


## Bernoulli 个体聚合后的检验样本要求

In [36]:
# 按 bernoulli 个体视角，采用卡方检验
p = 0.4
abs_mde = 0.04

prop_test_res = power.prop.test(p1 = p, p2 = p + abs_mde, sig.level = 0.1, power = 0.8, alternative = 'one.sided')
floor(prop_test_res$n)

[1] 1371

In [34]:
group_size = 100
p_std = (p * (1-p) / group_size)^0.5

t_test_res = power.t.test(delta = abs_mde / p_std, sig.level = 0.1, power = 0.8, alternative = 'one.sided')
floor(t_test_res$n * group_size)

[1] 1396

List of 8
 $ n          : num 14
 $ delta      : num 0.816
 $ sd         : num 1
 $ sig.level  : num 0.1
 $ power      : num 0.8
 $ alternative: chr "one.sided"
 $ note       : chr "n is number in *each* group"
 $ method     : chr "Two-sample t test power calculation"
 - attr(*, "class")= chr "power.htest"


In [41]:
power.prop.test(p1 = 0.65, p2 = 0.65 * 1.05, sig.level = .05, power = 0.8, alternative = "one.sided")


     Two-sample comparison of proportions power calculation 

              n = 2602.048
             p1 = 0.65
             p2 = 0.6825
      sig.level = 0.05
          power = 0.8
    alternative = one.sided

NOTE: n is number in *each* group
